# The Problem

We will create a model based on simple artifial neural networks (ANNs) each hand image to the corresponding number the hand is depicting ($0$ to $5$).

# Data Ingest

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import os


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [14]:
def construct_dataset_file(directory):
    col_list = ['label'] + list(np.arange(128*128))
    df = pd.DataFrame({ col: pd.Series(dtype='int') for col in col_list })
    for filename in os.listdir(directory):
        file_location = os.path.join(directory, filename)
        image = tf.keras.utils.load_img(file_location, color_mode='grayscale')
        pixels = np.array(image).flatten()
        
        label = int(filename[-6])
        df.loc[len(df.index)] = np.insert(pixels, 0, label)
        
    return df

def load_train_df(forced=False):
    TRAIN_IMAGES_DIRECTORY = '/kaggle/input/fingers/train'
    TRAIN_DATASET_FILE = '/kaggle/working/train.csv'

    if not os.path.exists(TRAIN_DATASET_FILE) or forced:
        print("Constructing train dataset")
        df = construct_dataset_file(TRAIN_IMAGES_DIRECTORY)
        df.to_csv(TRAIN_DATASET_FILE, index=False)
        return df
    else:
        print("Loading cached train dataset")
        return pd.read_csv(TRAIN_DATASET_FILE)

def load_test_df(forced=False):
    TEST_IMAGES_DIRECTORY = '/kaggle/input/fingers/test'
    TEST_DATASET_FILE = '/kaggle/working/test.csv'

    if not os.path.exists(TEST_DATASET_FILE) or forced:
        print("Constructing test dataset")
        df = construct_dataset_file(TEST_IMAGES_DIRECTORY)
        df.to_csv(TEST_DATASET_FILE, index=False)
        return df
    else:
        print("Loading cached test dataset")
        return pd.read_csv(TEST_DATASET_FILE)
    
# raw_train_df = load_train_df(forced=True)
# raw_test_df = load_test_df(forced=True)
raw_train_df, raw_test_df = load_train_df(), load_test_df()

Loading cached train dataset
Loading cached test dataset


In [20]:
# Let's have a look at the raw train dataset.
raw_train_df.head()

,label,0,1,2,3,4,5,6,7,8,...,16374,16375,16376,16377,16378,16379,16380,16381,16382,16383
0,4,58,56,54,54,54,53,49,48,51,...,34,34,42,47,50,52,52,51,51,51
1,4,52,50,47,44,41,41,43,43,41,...,41,40,41,39,33,26,21,19,18,18
2,0,37,35,33,33,36,36,33,34,40,...,33,31,32,30,26,29,39,42,41,40
3,4,51,52,53,53,52,50,49,50,54,...,45,46,42,41,43,45,50,50,47,45
4,1,67,66,65,66,68,70,72,72,71,...,60,58,56,55,53,54,56,57,58,58


In [10]:
# Let's have a look at the raw test dataset.
raw_test_df.head()

,label,0,1,2,3,4,5,6,7,8,...,16374,16375,16376,16377,16378,16379,16380,16381,16382,16383
0,1,67,66,64,65,67,66,64,64,66,...,68,68,67,63,55,52,54,55,55,55
1,0,78,75,71,69,69,64,56,55,62,...,66,65,63,63,66,66,65,65,66,67
2,0,58,56,53,54,61,63,61,59,57,...,63,63,63,64,66,68,71,70,66,65
3,3,51,51,53,53,53,53,54,55,56,...,67,63,56,53,52,50,46,46,48,49
4,4,34,34,36,37,39,40,41,43,46,...,35,34,31,32,38,38,34,30,28,26


In [11]:
def split_X_y(df):
    X = df.drop(columns=['label'])
    y = df['label']
    return X, y

# Split the train dataset
X_train, y_train = split_X_y(raw_train_df)
print(f"X_train shape: {X_train.shape}")
print(f"y_train length: {len(y_train)}")

# Split the test dataset
X_test, y_test = split_X_y(raw_test_df)
print(f"X_test shape: {X_test.shape}")
print(f"y_test length: {len(y_test)}")

X_train shape: (18000, 1)
y_train length: 18000


KeyError: 'image'

In [ ]:
# pd.DataFrame(raw_test_df['image'].tolist(), index=raw_test_df['image'])
# (raw_test_df['image'].to_list())
raw_test_df.image